This dataset is based on "Bank Marketing" 

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()
import os

pd.options.display.max_columns = None
from sklearn.preprocessing import LabelEncoder
import scipy.stats as stats
from scipy.stats import chi2_contingency
import warnings
import scikitplot as skplt
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [2]:
df = pd.read_csv('bank-additional-full.csv', sep=';')

In [3]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


Data Dictionary:
    Input variables:
   **bank client data**: 
   - age (numeric)
   - job : type of job (categorical: "admin.","blue-collar","entrepreneur","housemaid","management","retired","self-employed","services","student","technician","unemployed","unknown")
   - marital : marital status (categorical: "divorced","married","single","unknown"; note: "divorced" means divorced or widowed)
   - education (categorical: "basic.4y","basic.6y","basic.9y","high.school","illiterate","professional.course","university.degree","unknown")
   - default: has credit in default? (categorical: "no","yes","unknown")
   - housing: has housing loan? (categorical: "no","yes","unknown")
   - loan: has personal loan? (categorical: "no","yes","unknown")
   **related with the last contact of the current campaign**
   - contact: contact communication type (categorical: "cellular","telephone") 
   - month: last contact month of year (categorical: "jan", "feb", "mar", ..., "nov", "dec")
   - day_of_week: last contact day of the week (categorical: "mon","tue","wed","thu","fri")
   - duration: last contact duration, in seconds (numeric). Important note:  this attribute highly affects the output target (e.g., if duration=0 then y="no"). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
   **other attributes**:
   - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
   - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
   - previous: number of contacts performed before this campaign and for this client (numeric)
   - poutcome: outcome of the previous marketing campaign (categorical: "failure","nonexistent","success")
   **social and economic context attributes**:
   - emp.var.rate: employment variation rate - quarterly indicator (numeric)
   - cons.price.idx: consumer price index - monthly indicator (numeric)     
   - cons.conf.idx: consumer confidence index - monthly indicator (numeric)     
   - euribor3m: euribor 3 month rate - daily indicator (numeric)
   - nr.employed: number of employees - quarterly indicator (numeric)

  Output variable (desired target):
  21 - y - has the client subscribed a term deposit? (binary: "yes","no")

## Univariate Analsysis

In [ ]:
objs = df.select_dtypes('object')
objs = objs.dropna()
fig, ax = plt.subplots(11, 1, figsize=(10,50))
for i, obj in enumerate(objs):
    sns.countplot(obj, data=df, hue='y', orient='vertical', ax=ax[i], palette='viridis')
    for p in ax[i].patches:
        y=p.get_bbox().get_points()[1,1]
        ax[i].annotate('{:1.2f}%'.format(100.*y/df[obj].count()), (p.get_x()+0.2, p.get_height()+20), 
            ha='center', va='bottom',fontsize=10, color='red', rotation=20, xytext=(0, 10),
         textcoords='offset points') # set the alignment of the text

    ax[i].tick_params(axis='x', rotation=10)
    ax[i].set_title(obj)
    plt.tight_layout()

Based on above plots following assumptions can be made:


- more than 64% of the jobs belong to admin, blue-collar, technician
unemployed and students can be concatenated together and our maximum returns comes from this section only, in case of *job*

- almost 67% people were contacted over cellular and 9.35% success rate comes from the people who were contacted via cellular.

- more contacts were made in summer months, which shows us a direction that instead of taking 12 months, we can divide this into seasons. i.e summer winter and springs.

- in poutcome, almost 86% are nonexistent in the records, that is 86% are new poeple, we can again sgement people based on new customer s or old customers

Some other observations.
- unknowns corresponds to nulls, which are only few, so we can bear the loss of dropping nulls.
- illiterate in education is very rare (depending on the data), so we'll drop this.
- Nulls in education can be imputed.
- housing and personal loans has same amount of nulls/
- day of week is not showing any new information, so we'll drop this.
- our data is highly imbalenced.
- default is highly imbalanced, and has nulls, so to be on good side, let's drop this too!

In [ ]:
df['default'].value_counts()

In [ ]:
df.education.replace('illiterate', np.NaN, inplace=True)
df = df.drop(['day_of_week','default'], axis=1)

In [ ]:
df = df.replace('unknown', np.NaN)

In [ ]:
df.isnull().sum()/len(df)*100

For research point of view, we can impute nulls with using other techniquies like KNN Imputation, etc
but here, we can bear with loss of 7% of data.

In [ ]:
df.dropna(subset=['housing','job', 'education', 'marital', 'loan'], inplace=True)

In [ ]:
df['month'].value_counts()

In [ ]:
df.loc[df['month'].isin(['apr', 'may', 'jun', 'jul', 'aug']), 'season'] = 'summer'

In [ ]:
df.loc[df['month'].isin(['oct', 'nov',]), 'season'] = 'winter'

In [ ]:
df.loc[~df['month'].isin(['oct', 'nov', 'apr', 'may', 'jun', 'jul', 'aug']), 'season'] = 'thirdseason'

In [ ]:
df['season'].value_counts()

In [ ]:
nums = df.select_dtypes(['int64', 'float64'])

In [ ]:
fig = plt.figure(figsize=(12,10))
sns.heatmap(nums.corr(), cmap='viridis', annot=True)
plt.xticks(rotation=45)

Multicollinarity, we can again analyse the combinations of the data causing multicollinearity.
For example, euribor3m is collinear with nr.employed and emp.var.rate
We can drop or group them together, that affects the model for good

In [ ]:
df = df.drop('euribor3m', axis=1)

In [ ]:
nums.head()

Also, from the data dictionary; "he duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model."

So, we'll drop the duration as well

In [ ]:
df = df.drop('duration', axis=1)

Chi-Square test for testing collinearity in categorical data, to see, what features are important, what has minimum p-value

In [ ]:
class ChiSquare:
    def __init__(self, dataframe):
        self.df = dataframe
        self.p = None #P-Value
        self.chi2 = None #Chi Test Statistic
        self.dof = None
        
        self.dfObserved = None
        self.dfExpected = None
        
    def _print_chisquare_result(self, colX, alpha):
        result = ""
        if self.p<alpha:
            result=f"{colX} is IMPORTANT for Prediction with p-value = {self.p}"
        else:
            result="{0} is NOT an important predictor. (Discard {0} from model)".format(colX)

        print(result)
        
    def TestIndependence(self,colX,colY, alpha=0.05):
        X = self.df[colX].astype(str)
        Y = self.df[colY].astype(str)
        
        self.dfObserved = pd.crosstab(Y,X) 
        chi2, p, dof, expected = stats.chi2_contingency(self.dfObserved.values)
        self.p = p
        self.chi2 = chi2
        self.dof = dof 
        
        self.dfExpected = pd.DataFrame(expected, columns=self.dfObserved.columns, index = self.dfObserved.index)
        
        self._print_chisquare_result(colX,alpha)

In [ ]:
objs = df.select_dtypes('object'); objs.head()

In [ ]:
cst = ChiSquare(objs)
testColumns = objs.drop('y', axis=1)
for col in testColumns:
    cst.TestIndependence(colX=col, colY='y')

In [ ]:
df = df.drop(['housing', 'loan'], axis=1)

In [ ]:
df[objs.columns] = objs.apply(LabelEncoder().fit_transform)

In [ ]:
X_train, X_test, y_train, y_test =  train_test_split(df.drop('y', axis=1), df['y'], stratify=df['y'], test_size=0.2, random_state=42)

In [5]:
xgb = XGBClassifier(learning_rate=0.1, n_estimators=100)
xgb.fit(X_train, y_train)
xgb.score(X_test, y_test)

NameError: name 'X_train' is not defined

In [ ]:
y_pred = xgb.predict(X_test)
y_proba = xgb.predict_proba(X_test)

---

In [ ]:
from xgboost import plot_importance
fig , ax = plt.subplots(1,1,figsize=(12,10))
plot_importance(xgb, ax=ax)

So, our important features are the following:
    - nr.employed 
    - cons.conf.indx:
    - age:
    - month:
    - emp.var.rate:
    - campaign:
    - contact:
    - pdays:
    - poutcome:
    - cons.price.idx:
    - season:
    - job:
    - previous:
    - marital:

In [6]:
feats = {}
for feature, importance in zip(X_train.columns, xgb.feature_importances_):
    feats[feature] = importance
importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-importance'})
importances.sort_values(by='Gini-importance').plot(kind='bar', rot=45)


NameError: name 'X_train' is not defined

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

print('\n', classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import roc_curve, auc
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

In [7]:
skplt.metrics.plot_roc_curve(y_test, y_proba)

NameError: name 'y_test' is not defined